<a href="https://colab.research.google.com/github/sksabbir121/-Laravel-Project-CRUD/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow numpy nltk


In [ ]:
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Hey", "Howdy"],
      "responses": ["Hello!", "Hi there!", "Hey! How can I help you?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "See you", "Goodbye"],
      "responses": ["Goodbye!", "See you soon!", "Take care!"]
    },
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "That's helpful"],
      "responses": ["You're welcome!", "Happy to help!", "Anytime!"]
    }
  ]
}


{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hello', 'Hey', 'Howdy'],
   'responses': ['Hello!', 'Hi there!', 'Hey! How can I help you?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you', 'Goodbye'],
   'responses': ['Goodbye!', 'See you soon!', 'Take care!']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ["You're welcome!", 'Happy to help!', 'Anytime!']}]}

In [ ]:
# Load the dataset
import os
import json

file_path = 'intents.json'

if os.path.exists('intents.json'):
    with open('intents.json') as file:
        data = json.load(file)
else:
    print(f"Error: {file_path} not found. Please ensure the file is in the correct location.")

lemmatizer = WordNetLemmatizer()

# Prepare data
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and sort
words = sorted(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]))
classes = sorted(set(classes))

# Prepare training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = [lemmatizer.lemmatize(w.lower()) for w in doc[0]]
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

X = np.array([item[0] for item in training])
y = np.array([item[1] for item in training])


JSONDecodeError: Unexpected UTF-8 BOM (decode using utf-8-sig): line 1 column 1 (char 0)

In [ ]:
import os
import json

file_path = 'intents.json'

if os.path.exists('intents.json'):
    with open('intents.json', encoding='utf-8-sig') as file: # Open the file with utf-8-sig encoding
        data = json.load(file)
else:
    print(f"Error: {file_path} not found. Please ensure the file is in the correct location.")

# ... (rest of the code remains unchanged)

In [ ]:
# Define the model
model = Sequential([
    Dense(128, input_shape=(len(X[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(y[0]), activation='softmax')
])

# Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('chatbot_model.h5')
print("Model trained and saved.")


NameError: name 'Sequential' is not defined

In [ ]:
# Define the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

model = Sequential([
    Dense(128, input_shape=(len(X[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(y[0]), activation='softmax')
])

# Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('chatbot_model.h5')
print("Model trained and saved.")

NameError: name 'X' is not defined

In [ ]:
# Load the dataset
import os
import json
import nltk
import numpy as np
import random
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
# Download necessary NLTK resources if not already present
nltk.download('punkt')
nltk.download('wordnet')

# Load the dataset
import os
import json
import nltk
import numpy as np
import random
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
# Download necessary NLTK resources if not already present
nltk.download('punkt')
nltk.download('wordnet')

file_path = 'intents.json'

if os.path.exists('intents.json'):
    pass

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model

model = load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [1 if word in sentence_words else 0 for word in words]
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    for intent in intents_json['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

# Start the chatbot
print("Chatbot is running! Type 'quit' to exit.")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        print("Chatbot: Goodbye!")
        break
    ints = predict_class(message)
    res = get_response(ints, data)
    print(f"Chatbot: {res}")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'chatbot_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model
import os

# Check if the model file exists
if os.path.exists('chatbot_model.h5'):
    model = load_model('chatbot_model.h5')
else:
    print("Error: chatbot_model.h5 not found. Please train and save the model first.")
    # Train and save the model if it doesn't exist
    # ... (Your code to train and save the model) ...

    # Example:
    # from tensorflow.keras.models import Sequential
    # from tensorflow.keras.layers import Dense, Dropout
    # from tensorflow.keras.optimizers import SGD
    # # Define your model, compile it, train it, and then save it
    # # ...
    # model.save('chatbot_model.h5')
    # print("Model trained and saved.")

# ... (Rest of your chatbot code) ...

Error: chatbot_model.h5 not found. Please train and save the model first.


In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model
import os

# Check if the model file exists
if os.path.exists('chatbot_model.h5'):
    model = load_model('chatbot_model.h5')
else:
    print("Error: chatbot_model.h5 not found. Please train and save the model first.")
    # You might want to exit or handle this case differently based on your needs
    # For example, you could train the model here if it doesn't exist
    # ... code to train and save the model ...


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [1 if word in sentence_words else 0 for word in words]
```python
# Load the trained model
from tensorflow.keras.models import load_model
import os

# Check if the model file exists
if os.path.exists('chatbot_model.h5'):
    model = load_model('chatbot_model.h5')
else:
    print("Error: chatbot_model.h5 not found. Please train and save the model first.")
    # You might want to exit or handle this case differently based on your needs
    # For example, you could train the model here if it doesn't exist
    # ... code to train and save the model ...


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [1 if word in sentence_words else 0 for word in words]
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
```
```python
import os

# Check if the model file exists
if os.path.exists('chatbot_model.h5'):
    model = load_model('chatbot_model.h5')
else:
    print("Error: chatbot_model.h5 not found. Please train and save the model first.")
    # You might want to exit or handle this case differently based on your needs
    # For example, you could train the model here if it doesn't exist
    # ... code to train and save the model ...


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [1 if word in sentence_words else 0 for word in words]
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
```
import os

# Check if the model file exists
if os.path.exists('chatbot_model.h5'):
    model = load_model('chatbot_model.h5')
else:
    print("Error: chatbot_model.h5 not found. Please train and save the model first.")
    # You might want to exit or handle this case differently based on your needs
    # For example, you could train the model here if it doesn't exist
    # ... code to train and save the model ...


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [1 if word in sentence_words else 0 for word in words]
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
```
import os

# Check if the model file exists
if os.path.exists('chatbot_model.h5'):
    model = load_model('chatbot_model.h5')
else:
    print("Error: chatbot_model.h5 not found. Please train and save the model first.")
    # You might want to exit or handle this case differently based on your needs
    # For example, you could train the model here if it doesn't exist
    # ... code to train and save the model ...


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [1 if word in sentence_words else 0 for word in words]
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
```

SyntaxError: invalid syntax (<ipython-input-25-b390a433b63c>, line 23)